Learning how to use fileio in python
- the idea is get all files names in a folder
    - then for each bin(3 files, features.csv, autoclass.csv, .hdr file) do processing to remove, combine and write to one file
    - so get the D"year:month:day:hour:minute:second" timestamps and read in the 3 files

https://pythonguides.com/list-files-in-a-directory-with-python/

- There are multiple ways here are just a few that were listed

In [ ]:
from pathlib import Path
p = Path('./docs')
files = [f.name for f in p.iterdir() if f.is_file()]
print(files)

In [ ]:
import os

# Set the directory you want to start from
root_dir = './docs'

for dir_path, subdirs, files in os.walk(root_dir):
    for file in files:
        print(os.path.join(dir_path, file))

https://docs.python.org/3/library/glob.html

- testing how to get file names from a folder, and using that folder get the date for the next step

In [ ]:
import glob
import os

path = "./ifcb_downloads/202501/"

for files in glob.glob(path + '*.hdr'):
    date = os.path.basename(files).split('_')[0].lstrip('D')
    # print(date)

testing with just one date

In [2]:
from pathlib import Path
import pandas as pd
import glob
import os

out_dir = Path("./processed/")
out_dir.mkdir(parents=True, exist_ok=True)

path = "./ifcb_downloads/202501"
date = "20250101T185049"

# this one has an empty features file
# date = "20250115T091142"

def read_hdr(path: Path) -> dict:
    return { k.strip(): v.strip()
            for line in path.read_text().splitlines()
            if ":" in line
            for k, v in [line.split(":", 1)] }

matches = glob.glob(os.path.join(path, f"*{date}*"))

for match in matches:
    if match.endswith('.hdr'):
        # idk what do do with hdr file yet but I'll figure it out later
        hdr = read_hdr(Path(match))

        temperature = float(hdr["temperature"])
        humidity = float(hdr["humidity"])
    elif match.endswith('.csv'):
        if "_features" in match:
            try:
                features = pd.read_csv(Path(match))
                features.drop(columns=['roi_number'], inplace=True)
            except pd.errors.EmptyDataError:
                features = pd.DataFrame()
        else:
            class_scores = pd.read_csv(Path(match), usecols=['pid', 'Lingulodinium_polyedra'])

combined = pd.concat([class_scores, features], axis=1)
# combined["temperature"] = temperature
# combined["humidity"] = humidity

combined.head()

out_file = out_dir / f"{date}.csv"
# combined.to_csv(out_file, index=False)
print(f"Saved data to {out_file}")

Saved data to processed\20250101T185049.csv


testing iterating through month folders

In [9]:
from pathlib import Path

path = "./ifcb_downloads/"
path2 = Path(path)

for month_dir in sorted(path2.iterdir()):
    # print(month_dir.name)
    burger = path + month_dir.name + '/'
    print(burger)

./ifcb_downloads/202501/
./ifcb_downloads/202502/
./ifcb_downloads/202503/
./ifcb_downloads/202504/
./ifcb_downloads/202505/
./ifcb_downloads/202506/
./ifcb_downloads/202507/
./ifcb_downloads/202508/
./ifcb_downloads/202509/
./ifcb_downloads/202510/
./ifcb_downloads/202512/


adding something for hdr

In [10]:
from pathlib import Path

p = Path("./ifcb_downloads/202501/D20250101T185049_IFCB183.hdr")

def read_hdr(path: Path) -> dict:
    return {
        k.strip(): v.strip()
        for line in path.read_text().splitlines()
        if ":" in line
        for k, v in [line.split(":", 1)]
    }

hdr = read_hdr(p)

temperature = float(hdr["temperature"])
humidity = float(hdr["humidity"])
print(f"Temperature: {temperature}, Humidity: {humidity}")

Temperature: 27.712664225223172, Humidity: 4.007185473411155
